### 1. Fine-tuning 

References: 
- https://llama.meta.com/docs/how-to-guides/fine-tuning/
- https://github.com/MuhammadMoinFaisal/LargeLanguageModelsProjects/tree/main/Fine-Tune%20Llama%202

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

#### Login to Hugging Face Hub

In [ ]:
from huggingface_hub import notebook_login
notebook_login()  #enter your tokens

#### QLORA

In [ ]:
# The model that you want to train from the Hugging Face hub
model_name = "meta-llama/Llama-2-13b-chat-hf"
dataset_path = "Path to your repo in the Hugging Face Hub where training and validation datasets are stored (e.g., myname/datasets)."
new_model = "Your fine-tuned model name (e.g., llama-us-ccss)"

#QLORA
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1

use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

output_dir = "./results"

# Number of training epochs
num_train_epochs = 3

fp16 = False
bf16 = False

per_device_train_batch_size = 2
per_device_eval_batch_size = 2
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3


learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "cosine"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 0
logging_steps = 25

max_seq_length = None
packing = False
# Load the entire model on the GPU 0
device_map = {"": 0}

#### File Uploads

In [ ]:
data_files_train = {"train": "Your training file (e.g., data/examples/llama_finetuning_train.csv)"}
df_train = load_dataset(dataset_path, data_files=data_files_train, split="train")

data_files_val = {"validation": "Your validation file (e.g., data/examples/llama_finetuning_val.csv)"}
df_val = load_dataset(dataset_path, data_files=data_files_val, split="validation")

#### Fine-tuning

In [ ]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)


if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)


model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training


peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
    )


trainer = SFTTrainer(
    model=model,
    train_dataset=df_train,
    eval_dataset=df_val,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)


trainer.train()

trainer.model.save_pretrained(new_model)

In [ ]:
# Empty VRAM
del model
del pipe
del trainer
import gc
gc.collect()
gc.collect()

#### Store your new fine-tuned model

In [ ]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    #device_map=device_map,  # cuda out of memory
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

#### Push the model to Hugging Face Hub

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

finetuned_model_path="Path to your repo in the Hugging Face Hub where your new fine-tuned model will be stored (e.g., myname/fine-tuning)"

model.push_to_hub(finetuned_model_path, check_pr=True)
tokenizer.push_to_hub(finetuned_model_path,check_pr=True)

#### Inference

In [ ]:
import torch
import transformers
from torch import cuda, bfloat16

In [ ]:
model_id=finetuned_model_path
hf_auth="Your tokens"
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(device)
device_name=torch.cuda.get_device_name(0)
print(f"Using device:{device} ({device_name})")

bnb_config=transformers.BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_use_double_quant=True,
                                          bnb_4bit_compute_dtype=bfloat16)

model_config=transformers.AutoConfig.from_pretrained(model_id, use_auth_token=hf_auth)

model=transformers.AutoModelForCausalLM.from_pretrained(model_id,
                                                       trust_remote_code=True,
                                                       config=model_config,
                                                       quantization_config=bnb_config,
                                                       use_auth_token=hf_auth)

model.eval()
print(f"Model loaded on {device}")

tokenizer=transformers.AutoTokenizer.from_pretrained(model_id, use_auth_token=hf_auth)

In [ ]:
csv_file_path = "Path to an input file containing the problems"  
pred_csv_file_path="Path to an output file containing the problems and skill predictions"

problems = []
with open(csv_file_path, 'r', encoding='utf-8') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        problems.append(row['problem'])  
        
generate_text=transformers.pipeline(model=model,
                                   tokenizer=tokenizer,
                                   return_full_text=True,
                                   task='text-generation',
                                   temperature=0.2,
                                   max_new_tokens=250,
                                   repetition_penalty=1.1)

predicted_skills = []

for problem_text in problems:
    prompt = f"""
Please provide one standard from the Common Core Standards for Mathematics that is most closely related to the given problem. Problem: "{problem_text}"
"""
    prompt_template=f'''[INST] <<SYS>>


    {prompt} [/INST]
 
    '''
    res=generate_text(prompt)
    output_text=res[0]["generated_text"]
    
      
    predicted_skill = output_text
    
    if predicted_skill:
        predicted_skills.append(predicted_skill)
    else:
        predicted_skills.append('')

updated_rows = []
with open(csv_file_path, 'r', encoding='utf-8') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    fieldnames = csv_reader.fieldnames + ['pred_skill']
    for row, predicted_skill in zip(csv_reader, predicted_skills):
        row['pred_skill'] = predicted_skill
        updated_rows.append(row)

with open(pred_csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    csv_writer.writeheader()
    csv_writer.writerows(updated_rows)  

### 2. Standards in the prompt

In [ ]:
model_id="meta-llama/Llama-2-13b-chat-hf"
hf_auth="Your tokens"
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.cuda.set_device(device)
device_name=torch.cuda.get_device_name(0)
print(f"Using device:{device} ({device_name})")

bnb_config=transformers.BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type="nf4", bnb_4bit_use_double_quant=True,
                                          bnb_4bit_compute_dtype=bfloat16)

model_config=transformers.AutoConfig.from_pretrained(model_id, use_auth_token=hf_auth)

model=transformers.AutoModelForCausalLM.from_pretrained(model_id,
                                                       trust_remote_code=True,
                                                       config=model_config,
                                                       quantization_config=bnb_config,
                                                       use_auth_token=hf_auth)

model.eval()
print(f"Model loaded on {device}")

tokenizer=transformers.AutoTokenizer.from_pretrained(model_id, use_auth_token=hf_auth)

In [ ]:
csv_file_path = "Path to an input file containing the problems"  
pred_csv_file_path="Path to an output file containing the problems and skill predictions"
standards="A list of standards (skills)"

problems = []
with open(csv_file_path, 'r', encoding='utf-8') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    for row in csv_reader:
        problems.append(row['problem'])  
        
generate_text=transformers.pipeline(model=model,
                                   tokenizer=tokenizer,
                                   return_full_text=True,
                                   task='text-generation',
                                   temperature=0.2,
                                   max_new_tokens=250,
                                   repetition_penalty=1.1)

predicted_skills = []

for problem_text in problems:
    prompt = f"""
Please provide one standard from the Common Core Standards for Mathematics that is most closely related to the given problem. Problem: "{problem_text}", Standards: {standards}
"""
    prompt_template=f'''[INST] <<SYS>>


    {prompt} [/INST]
 
    '''
    res=generate_text(prompt)
    output_text=res[0]["generated_text"]
    
    predicted_skill = output_text
    
    if predicted_skill:
        predicted_skills.append(predicted_skill)
    else:
        predicted_skills.append('')

updated_rows = []
with open(csv_file_path, 'r', encoding='utf-8') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    fieldnames = csv_reader.fieldnames + ['pred_skill']
    for row, predicted_skill in zip(csv_reader, predicted_skills):
        row['pred_skill'] = predicted_skill
        updated_rows.append(row)

with open(pred_csv_file_path, 'w', newline='', encoding='utf-8') as csv_file:
    csv_writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    csv_writer.writeheader()
    csv_writer.writerows(updated_rows)  